In [180]:
from keras.layers import Input, Dense, Dropout, Concatenate, Add
from keras.models import Model
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard
from __future__ import print_function
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn import preprocessing

import keras
import matplotlib.pyplot as plt
import numpy as np
import math
import pydot
import graphviz
import pandas as pd

# Data Set Information

This radar data was collected by a system in Goose Bay, Labrador. This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts. See the paper for more details. The targets were free electrons in the ionosphere. "Good" radar returns are those showing evidence of some type of structure in the ionosphere. "Bad" returns are those that do not; their signals pass through the ionosphere. 

Received signals were processed using an autocorrelation function whose arguments are the time of a pulse and the pulse number. There were 17 pulse numbers for the Goose Bay system. Instances in this databse are described by 2 attributes per pulse number, corresponding to the complex values returned by the function resulting from the complex electromagnetic signal.

## Attribute Information

- All 34 are continuous 
- The 35th attribute is either "good" or "bad" according to the definition summarized above. This is a binary classification task. 
- https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.names

## Data Import and preprocessing

In [126]:
data = np.genfromtxt("data/ionosphere.data")

In [190]:
data = pd.read_csv('data/ionosphere.data', sep=",", header=None)
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
count,351.000000,351.0,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,...,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,0.891738,0.0,0.641342,0.044372,0.601068,0.115889,0.550095,0.119360,0.511848,0.181345,...,0.396135,-0.071187,0.541641,-0.069538,0.378445,-0.027907,0.352514,-0.003794,0.349364,0.014480
std,0.311155,0.0,0.497708,0.441435,0.519862,0.460810,0.492654,0.520750,0.507066,0.483851,...,0.578451,0.508495,0.516205,0.550025,0.575886,0.507974,0.571483,0.513574,0.522663,0.468337
min,0.000000,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,0.0,0.472135,-0.064735,0.412660,-0.024795,0.211310,-0.054840,0.087110,-0.048075,...,0.000000,-0.332390,0.286435,-0.443165,0.000000,-0.236885,0.000000,-0.242595,0.000000,-0.165350
50%,1.000000,0.0,0.871110,0.016310,0.809200,0.022800,0.728730,0.014710,0.684210,0.018290,...,0.553890,-0.015050,0.708240,-0.017690,0.496640,0.000000,0.442770,0.000000,0.409560,0.000000
75%,1.000000,0.0,1.000000,0.194185,1.000000,0.334655,0.969240,0.445675,0.953240,0.534195,...,0.905240,0.156765,0.999945,0.153535,0.883465,0.154075,0.857620,0.200120,0.813765,0.171660
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [182]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
count,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,...,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,0.891738,0.820671,0.522186,0.800534,0.557945,0.775048,0.559680,0.755924,0.590673,0.738091,...,0.464407,0.770820,0.465231,0.689223,0.486046,0.676257,0.498103,0.674682,0.507240,0.641026
std,0.311155,0.248854,0.220717,0.259931,0.230405,0.246327,0.260375,0.253533,0.241925,0.281748,...,0.254247,0.258102,0.275013,0.287943,0.253987,0.285742,0.256787,0.261332,0.234169,0.480384
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.736067,0.467633,0.706330,0.487603,0.605655,0.472580,0.543555,0.475962,0.510560,...,0.333805,0.643217,0.278417,0.500000,0.381557,0.500000,0.378702,0.500000,0.417325,0.000000
50%,1.000000,0.935555,0.508155,0.904600,0.511400,0.864365,0.507355,0.842105,0.509145,0.833990,...,0.492475,0.854120,0.491155,0.748320,0.500000,0.721385,0.500000,0.704780,0.500000,1.000000
75%,1.000000,1.000000,0.597093,1.000000,0.667328,0.984620,0.722838,0.976620,0.767097,0.978948,...,0.578383,0.999972,0.576767,0.941732,0.577038,0.928810,0.600060,0.906883,0.585830,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [194]:
data.drop(data.columns[1], inplace=True, axis=1)
data[34] = [1 if e is "g" else 0 for e in data[34]]
data.head()

,0,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,1,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


In [195]:
# sample the dataframe
data_train = data.sample(frac=0.9, random_state=42)
data_valid = data.drop(data_train.index)

In [196]:
df_x_train = data_train.iloc[:,:-1]
df_x_train = df_x_train.transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df_y_train = data_train.iloc[:,-1]

df_x_valid = data_valid.iloc[:,:-1]
df_x_valid = df_x_valid.transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df_y_valid = data_valid.iloc[:,-1]

In [209]:
df_x_train.describe()

,0,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
count,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,...,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000,316.000000
mean,0.895570,0.817784,0.522094,0.796498,0.563261,0.773132,0.562843,0.755772,0.596871,0.729752,...,0.688157,0.465685,0.759911,0.465155,0.682221,0.488292,0.665222,0.494916,0.662627,0.507965
std,0.306303,0.251818,0.222290,0.262939,0.233777,0.246856,0.266824,0.249016,0.243454,0.286458,...,0.292496,0.258101,0.262799,0.278669,0.288899,0.257573,0.288393,0.260966,0.261653,0.234951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.726011,0.464336,0.700079,0.488350,0.613897,0.471016,0.542396,0.476426,0.504012,...,0.500000,0.321889,0.628716,0.263979,0.500000,0.380319,0.500000,0.376049,0.500000,0.422180
50%,1.000000,0.936722,0.510012,0.902620,0.514775,0.863272,0.510335,0.834157,0.512105,0.824230,...,0.759008,0.492542,0.837063,0.489053,0.731227,0.500000,0.710308,0.500000,0.690685,0.500000
75%,1.000000,1.000000,0.606244,1.000000,0.678753,0.984088,0.745698,0.977925,0.793609,0.981344,...,0.948491,0.581852,0.999959,0.581416,0.942238,0.582359,0.920661,0.601942,0.887807,0.589070
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [133]:
df_y_train.sum()/len(df_y_train)

0.6360759493670886

## preprocessing

In [207]:
x_train = np.array(df_x_train.as_matrix())
y_train = np.array(pd.DataFrame(df_y_train).as_matrix())

x_val = np.array(df_x_valid.as_matrix())
y_val = np.array(pd.DataFrame(df_y_valid).as_matrix())

In [200]:
input_dim = x_train.shape[1]

## Train Classifier

The goal is to get a very accurate classifier.

In [208]:
result = []
for i in range(1,5):
    model = Sequential()
    model.add(Dense(15, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(15, activation='relu'))
    model.add(Dropout(0.1))
#     model.add(Dense(15, activation='relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train, 
              epochs=150,
              shuffle=True,
              batch_size=4,
              verbose=0,
              validation_split=0.1
             )
    score = model.evaluate(x_val, y_val)[1]
    print(score)
    result.append(score)
print(np.mean(result))

35/35 [==============================] - 0s 129us/step
0.857142857143
35/35 [==============================] - 0s 157us/step
0.942857142857
35/35 [==============================] - 0s 114us/step
0.914285714286
35/35 [==============================] - 0s 129us/step
0.914285714286
0.907142857143


## Single Hidden Layer Autoencoder

In [166]:
result = []
for i in range(1,5):
    single_auto = Sequential()
    single_auto.add(Dense(8, input_dim=input_dim, activation='relu'))
    # model.add(Dropout(0.1))
    single_auto.add(Dense(input_dim, activation='sigmoid'))

    single_auto.compile(loss='mean_squared_error',
                  optimizer=RMSprop())

    single_auto.fit(x_train, x_train, 
                      epochs=150,
                      shuffle=True,
                      batch_size=4,
                      verbose=0,
                      validation_split=0.1
                     )
    score = single_auto.evaluate(x_val, x_val)
    result.append(score)
print(np.mean(result))

35/35 [==============================] - 0s 100us/step
0.116469529812


How well does a classifier after the vanilla autoencoder perform?

In [165]:
model.evaluate(single_auto.predict(x_val), y_val)

35/35 [==============================] - 0s 172us/step


[1.4033254964027557, 0.80000000000000004]

So we lose about 12% by decoding

# Stacked Autoencoder

In [160]:
######## constants for stacked autoencoder ############
encoding_dim1 = 16
encoding_dim2 = 8
decoding_dim1 = 16
decoding_dim2 = input_dim

In [162]:
result = []
for i in range(1,5):
    stacked_auto = Sequential()
    stacked_auto.add(Dense(encoding_dim1, input_dim=input_dim, activation='relu'))
    # model.add(Dropout(0.1))
    stacked_auto.add(Dense(encoding_dim2, activation='relu'))
    # model.add(Dropout(0.05))
    stacked_auto.add(Dense(decoding_dim1, activation='relu'))
    # model.add(Dropout(0.05))
    stacked_auto.add(Dense(decoding_dim2, activation='sigmoid'))

    stacked_auto.compile(loss='mean_squared_error',
                  optimizer=RMSprop())

    stacked_auto.fit(x_train, x_train, 
                      epochs=150,
                      shuffle=True,
                      batch_size=4,
                      verbose=0,
                      validation_split=0.1
                     )
    score = stacked_auto.evaluate(x_val, x_val)
    result.append(score)
print(np.mean(score))

35/35 [==============================] - 0s 100us/step
0.130945818326


In [163]:
model.evaluate(stacked_auto.predict(x_val), y_val)

35/35 [==============================] - 0s 0us/step


[0.43043743185698985, 0.82857142857142863]

# Experiment

In [179]:
input_data = Input(shape=(input_dim,))
encoder = Dense(8, activation='relu')(input_data)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
# concat = Add()[encoder, decoder]
classifier = Dense(1, activation='sigmoid')(decoder)

exp = Model(input_data, classifier)
exp.compile(loss='binary_crossentropy',
            optimizer=RMSprop(),
            metrics=['accuracy'])

exp.fit(x_train, y_train, 
          epochs=150,
          shuffle=True,
          batch_size=4,
          verbose=0,
          validation_split=0.1
         )
score = exp.evaluate(x_val, y_val)
print(score)

35/35 [==============================] - 0s 100us/step
[0.58571770218404706, 0.91428571428571426]
